# Sequence Classification using Recurrent Neural Networks(RNN)
In this homework, you will learn how to train a recurrent neural network for human action classification. RNN is designed handle sequential data. The network can incorporate both past history and current input. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) is a very good tutorial. You should read it before you start.

## Setup
Please make sure you have h5py and torchnet installed
> pip install h5py

> pip install git+https://github.com/pytorch/tnt.git@master


In [108]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


use cuda: False


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25).

For your convenience, we provide the dataloader for you.


In [109]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('./data', subset='train')
valSet = Dataset('./data', subset='val')
tstSet = Dataset('./data', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=2, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=1, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=1, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

## Model
Pytorch has implemented different types of recurrent layers for you. For this homework, you can use any type of RNNs as you want:
> torch.nn.RNN()

> torch.nn.LSTM()

> torch.nn.GRU()

You can check details for different types of recurrent layers here: [RNN](http://pytorch.org/docs/master/nn.html#torch.nn.RNN), [LSTM]( http://pytorch.org/docs/master/nn.html#torch.nn.LSTM), [GRU](http://pytorch.org/docs/master/nn.html#torch.nn.GRU)


### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.



In [110]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
        self.project_layer = nn.Linear(75, 100)
        self.recurrent_layer = nn.LSTM(100, 100, 1)
        self.classify_layer = nn.Linear(100, 10)
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SequenceClassify()

## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [111]:
################ 2nd To Do  (5 points)##################
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)
criterion = torch.nn.CrossEntropyLoss()

In [112]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 1
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


/Users/manideepattanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  , Loss: 2.3141,  Accuracy: 9.65
val Epoch: 0  , Loss: 2.3148,  Accuracy: 8.50


## Submit your results on Kaggle

### Train a better model for action recognition!
Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves better accuracy on the action recognition validation set.


### Testing the model and submit on Kaggle
Testing the model on the testing set and save the results as a .csv file. 
Please submitted the results.csv file generated by predict_on_test() to Kaggle(https://www.kaggle.com/t/934b80879bd741e6ac1967195604d4d9) to see how well your network performs on the test set. 
################ 3rd To Do  (20 points, the highest 3 entries get extra 10 points) ###############


In [145]:
# Use your best model to generate results on test set.
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size()
        return x.view(N, -1)

class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()

        self.convolution_layer = nn.Sequential(
            nn.Conv2d(1, 200, kernel_size=(3, 3), stride=1),
            nn.MaxPool2d(2, 2),
            nn.LeakyReLU(0.02),
            nn.BatchNorm2d(200),
#             nn.Dropout(0.4),
            nn.Conv2d(200, 100, 2, 1),
            nn.MaxPool2d(2, 2),
            nn.LeakyReLU(0.02),
            nn.BatchNorm2d(100),
#             nn.Dropout(0.4),
            Flatten(),
            nn.Linear(3400, 100)
        )
        self.recurrent_layer = nn.LSTM(100, 100, 1)
        self.project_layer = nn.Linear(100, 10)
    
    def forward(self, input, h_t_1=None, c_t_1=None):
        rnn_outputs = self.convolution_layer(input.view(50, 1, 15, 75))
        rnn_outputs, (hn, cn) = self.recurrent_layer(rnn_outputs.view(50, 1, 100))        
        logits = self.project_layer(rnn_outputs[:,-1])
        return logits
    
model = SequenceClassify()

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 100
evaluate_every_epoch = 1
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)

/Users/manideepattanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  , Loss: 2.1070,  Accuracy: 20.95
val Epoch: 0  , Loss: 1.8186,  Accuracy: 38.50
train Epoch: 1  , Loss: 1.4466,  Accuracy: 51.40
val Epoch: 1  , Loss: 1.2907,  Accuracy: 56.25
train Epoch: 2  , Loss: 1.0437,  Accuracy: 65.70
val Epoch: 2  , Loss: 0.9459,  Accuracy: 70.25
train Epoch: 3  , Loss: 0.8888,  Accuracy: 69.95
val Epoch: 3  , Loss: 0.8002,  Accuracy: 75.75
train Epoch: 4  , Loss: 0.8001,  Accuracy: 73.20
val Epoch: 4  , Loss: 0.7241,  Accuracy: 77.00
train Epoch: 5  , Loss: 0.7236,  Accuracy: 75.35
val Epoch: 5  , Loss: 0.7462,  Accuracy: 75.00
train Epoch: 6  , Loss: 0.6647,  Accuracy: 77.35
val Epoch: 6  , Loss: 0.6847,  Accuracy: 77.50
train Epoch: 7  , Loss: 0.6189,  Accuracy: 79.85
val Epoch: 7  , Loss: 0.7134,  Accuracy: 76.25
train Epoch: 8  , Loss: 0.5867,  Accuracy: 80.20
val Epoch: 8  , Loss: 0.5798,  Accuracy: 79.75
train Epoch: 9  , Loss: 0.5203,  Accuracy: 82.55
val Epoch: 9  , Loss: 0.5954,  Accuracy: 79.75
train Epoch: 10  , Loss: 0.4757,  Accura

train Epoch: 84  , Loss: 0.0258,  Accuracy: 99.30
val Epoch: 84  , Loss: 0.6190,  Accuracy: 84.25
train Epoch: 85  , Loss: 0.0213,  Accuracy: 99.60
val Epoch: 85  , Loss: 0.5096,  Accuracy: 87.75
train Epoch: 86  , Loss: 0.0161,  Accuracy: 99.55
val Epoch: 86  , Loss: 0.5562,  Accuracy: 88.75
train Epoch: 87  , Loss: 0.0090,  Accuracy: 99.90
val Epoch: 87  , Loss: 0.5720,  Accuracy: 88.50
train Epoch: 88  , Loss: 0.0200,  Accuracy: 99.50
val Epoch: 88  , Loss: 0.5564,  Accuracy: 87.50
train Epoch: 89  , Loss: 0.0079,  Accuracy: 99.90
val Epoch: 89  , Loss: 0.5374,  Accuracy: 89.00
train Epoch: 90  , Loss: 0.0149,  Accuracy: 99.45
val Epoch: 90  , Loss: 0.5502,  Accuracy: 85.75
train Epoch: 91  , Loss: 0.0061,  Accuracy: 100.00
val Epoch: 91  , Loss: 0.5432,  Accuracy: 87.75
train Epoch: 92  , Loss: 0.0034,  Accuracy: 100.00
val Epoch: 92  , Loss: 0.5617,  Accuracy: 89.00
train Epoch: 93  , Loss: 0.0024,  Accuracy: 100.00
val Epoch: 93  , Loss: 0.5585,  Accuracy: 88.50
train Epoch: 94  

In [146]:
# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results5.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i])+'\n')
            count+=1
    results.close()
    return count

print(tstLD)
count=predict_on_test(model, tstLD)
print(count)

1000


## Report the performance
################ 4th To Do  (5 points)##################

In this cell, you should write an explanation of what you did (network architecture, optimiziter, learning rate, epochs) and any visualizations or graphs that you make in the process of training and evaluating your network.

Report your Kaggle Performance here:

Network Architecture:
Used 3 layers in sequence. The first one is Convolution layer. The second layer is LSTM layer and the third layer is a project layer which maps the final output of the LSTM layer to what is required.
Convolution layer has following layers:
1. 3X3 convolution layer with 200 filters and stride 1
2. Max Pool layer with size 2X2 and stride 2
3. LeakyReLU layer with alpha=0.02
4. Batch Normalization
5. 3X3 convolution layer with 100 filters and stride 1
6. Max Pool layer with size 2X2 and stride 2
7. LeakyReLU layer with alpha=0.02
8. Batch Normalization
9. Flatten
10. Affine Layer to reduce inputs from 100 to 10.

Optimizer, learning rate and epochs:
Used Adam optimizer with a learning rate of 1e-3 and cross entropy loss as loss function. Ran for 100 epochs and validation accuracy at the end was around 87%. 

Kaggle accuracy: 80.8%